<a href="https://colab.research.google.com/github/golivetti/Diplomatura-DL2020/blob/main/Laboratorio_TNG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Discreto

Haremos un clasificador de artículos utilizando un modelo de Naive Bayes discreto. Trabajaremos con el dataset de Twenty News Group. Antes de empezar carguemos el dataset:

In [ ]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=False)

In [ ]:
twenty_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
len(twenty_train.data) #Cantidad de artículos periodísticos

11314

In [ ]:
set(twenty_train["target"]) #Clasificaciones de los artículos

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

In [ ]:
twenty_train["target_names"] #Referencia de los números de target

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
twenty_train.data[0] # Primer artículo

"From: cubbie@garnet.berkeley.edu (                               )\nSubject: Re: Cubs behind Marlins? How?\nArticle-I.D.: agate.1pt592$f9a\nOrganization: University of California, Berkeley\nLines: 12\nNNTP-Posting-Host: garnet.berkeley.edu\n\n\ngajarsky@pilot.njin.net writes:\n\nmorgan and guzman will have era's 1 run higher than last year, and\n the cubs will be idiots and not pitch harkey as much as hibbard.\n castillo won't be good (i think he's a stud pitcher)\n\n       This season so far, Morgan and Guzman helped to lead the Cubs\n       at top in ERA, even better than THE rotation at Atlanta.\n       Cubs ERA at 0.056 while Braves at 0.059. We know it is early\n       in the season, we Cubs fans have learned how to enjoy the\n       short triumph while it is still there.\n"

Vamos a aplicar el siguiente procesamiento utilizando los conceptos vistos en clase:

* Tokenization (nltk)
* Lemmatization (nltk)
* Stop Words (nltk)
* Stemming (nltk)
* Filtrado de palabras
* Obtención del vocabulario (countvectorizer)
* Transformación de los artículos en vectores
* Armado del modelo de Naive Bayes Multinomial
* Evaluación con el Train Set
* Evaluación con el Test Set

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Procesando todos los artículos:
articulos_procesados=list()
for idx in range(len(twenty_train.data)):
    if idx%1000==0:
        print(f'se procesaron {idx} artículos')
    art=twenty_train.data[idx]
    tok=word_tokenize(art)
    lem=[lemmatizer.lemmatize(x,pos='v') for x in tok]
    stop = [x for x in lem if x not in stopwords.words('english')]
    stem=[stemmer.stem(x) for x in stop]
    alpha=[x for x in stem if x.isalpha()]
    articulos_procesados.append(" ".join(alpha))

se procesaron 0 artículos
se procesaron 1000 artículos
se procesaron 2000 artículos
se procesaron 3000 artículos
se procesaron 4000 artículos
se procesaron 5000 artículos
se procesaron 6000 artículos
se procesaron 7000 artículos
se procesaron 8000 artículos
se procesaron 9000 artículos
se procesaron 10000 artículos
se procesaron 11000 artículos


In [ ]:
# Extracting features from articles
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_df=0.6,min_df=10) #chequear que max_df y min_df sean los que se piden
count_vect.fit(articulos_procesados) #Aprende el vocabulario y le asigna un código a cada palabra

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.6, max_features=None, min_df=10,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

En la siguiente celda de código transforme los artículos procesados al vector de cuentas de palabras, es decir, transforme los artículos procesados utilizando el count vectorizer.

In [ ]:
X_train_data=count_vect.transform(articulos_procesados)

Utilice la función MultinomialNB de sklear para implementar un clasificador Naive Bayes discreto. Utilice smoothing laplaciano con alpha=3.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=3)
clf.fit(X_train_data.toarray(),twenty_train['target'])

MultinomialNB(alpha=3, class_prior=None, fit_prior=True)

## Evaluación con el train set
Evalúe el accuracy del modelo entrenado utilizando el train set.

In [ ]:
clf.score(X_train_data.toarray(), twenty_train['target'])

0.9129397207000177

# Evaluación con el test set
Procese y convierta los artículos del test-set. Evalúe el accuracy del modelo con los parámetros obtenidos anteriormente utilizando el test-set.

In [ ]:
twenty_test = fetch_20newsgroups(subset='test', shuffle=False)

In [ ]:
#Procesando todos los artículos:
articulos_procesados_test=list()
for idx in range(len(twenty_test.data)):
    if idx%1000==0:
        print(f'se procesaron {idx} artículos')
    art=twenty_test.data[idx]
    tok=word_tokenize(art)
    lem=[lemmatizer.lemmatize(x,pos='v') for x in tok]
    stop = [x for x in lem if x not in stopwords.words('english')]
    stem=[stemmer.stem(x) for x in stop]
    alpha=[x for x in stem if x.isalpha()]
    articulos_procesados_test.append(" ".join(alpha))

se procesaron 0 artículos
se procesaron 1000 artículos
se procesaron 2000 artículos
se procesaron 3000 artículos
se procesaron 4000 artículos
se procesaron 5000 artículos
se procesaron 6000 artículos
se procesaron 7000 artículos


In [ ]:
#Transforme los artículos de test procesados
X_test_data=count_vect.transform(articulos_procesados_test)

In [ ]:
#Evalúe el score del modelo entrenado para el train set para los artículos de test
clf.score(X_test_data.toarray(), twenty_test['target'])

0.7841210833775889

In [ ]:
X_test_data.shape

(7532, 9139)